# 4.10 Coding Etiquette & Excel Reporting

## Points of this Script
1. Imports
2. Security Implications
3. Regional Segmentation
4. Regional Spending Habits
5. Customer Activity
6. Merge Data
7. Department Sales
8. Customer Profiles
9. Exports

### 1. Imports

In [ ]:
# Import library

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [ ]:
# Import dataframe

path = r'C:\Users\walls\Documents\Coding\Data Analysis\CareerFoundry\Data Immersion A4\Instacart Basket Analysis 01-25'
df_final = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'customer_merged.pkl'))

In [ ]:
df_final.head()

In [ ]:
df_final.shape

### 2. Security Implications

##### Consider any security implications that might exist for df_final

This data set does contain PII because of its first_name, surname, and state columns that give traceable and personal information about customers.
Some of these columns may need to be scrubbed to protect customer privacy.

In [ ]:
# Removing first_name and surname columns for data privacy
df_final.drop(['first_name', 'surname'], axis=1, inplace=True)

In [ ]:
df_final.head()

In [ ]:
df_final.shape

### 3. Regional Segmentation

In [ ]:
# Create data dictionary of states and regions

region = {
    "Northeast": ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'New Jersey', 'Pennsylvania'],
    "Midwest": ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri'],
    "South": ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana'],
    "West": ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']
}

In [ ]:
# Create if-statement with for-loop to assign state regions and create 'region' column

for region, states in region.items():
    df_final.loc[df_final['state'].isin(states), 'region'] = region

In [ ]:
df_final.head()

In [ ]:
df_final.shape

In [ ]:
df_final['region'].value_counts()

In [ ]:
# Check for missing values

df_final['region'].isnull().sum()

##### Observations:
1. region column created
2. states labeled with a region title
3. no missing values

### 4. Regional Spending Habits

In [ ]:
# Crosstab Region to spending flag
crosstab = pd.crosstab(df_final['region'], df_final['spending_flag'], dropna = False)

In [ ]:
crosstab.to_clipboard()

In [ ]:
sorted_crosstab = crosstab.sort_values(by=crosstab.columns.tolist(), ascending=False)

In [ ]:
# Create bar chart of crosstab table

region_spending_bar = sorted_crosstab.plot(kind='bar')
plt.title('Regional Spending Habits')
plt.xlabel('Region')
plt.ylabel('Spending Amount')
plt.ylim(100000, 12000000)
plt.xticks(rotation=0)

In [ ]:
# Export region chart
region_spending_bar.figure.savefig(os.path.join(path, 'Analysis','Visualizations', 'region_spending_bar.png'))

##### Observations
1. Low-spender far out-weigh high-spenders
2. Southern region has the highest low and high spenders
3. Northeast has the lowest low and high spenders

### 5. Customer Activity

In [ ]:
# Create exclusion_flag based on activity level

df_final.loc[df_final['max_order'] < 5, 'exclusion_flag'] = 'Low activity'
df_final.loc[df_final['max_order'] > 5, 'exclusion_flag'] = 'High activity'

In [ ]:
df_final['exclusion_flag'].value_counts(dropna = False)

##### Observations: 
1. More users are placing 5 or more orders than not

In [ ]:
df_final.head()

In [ ]:
df_final.shape

In [ ]:
# Remove low activity customers 

df_final_exclusion = df_final[df_final['exclusion_flag']== 'High activity']

In [ ]:
df_final_exclusion.head()

In [ ]:
df_final_exclusion.shape

In [ ]:
df_final_exclusion['exclusion_flag'].value_counts(dropna = False)

In [ ]:
# Export new df

df_final_exclusion.to_pickle(os.path.join(path, 'Data','Prepared Data', 'low_activity_excluded.pkl'))

### 6. Merge Data

In [ ]:
# Import new df

df_dep = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'departments_wrangled.csv'), index_col = False)

In [ ]:
df_dep.head()

In [ ]:
df_final_exclusion.columns

In [ ]:
df_dep.columns

In [ ]:
df_final_exclusion['department_id'].dtype

In [ ]:
df_dep['department_id'].dtype

In [ ]:
# Match data types

df_dep['department_id'] = df_dep['department_id'].astype('int16')

In [ ]:
# Merge df_final_exclusion with df_dep

df_final_merged = df_final_exclusion.merge(df_dep, on='department_id', indicator='merged') 

In [ ]:
df_final_merged.head()

In [ ]:
df_final_merged.shape

### 7. Department Sales

In [ ]:
# Create bar chart of prods orderd

dep_bar = df_final_merged['department'].value_counts(ascending=True).plot(kind='barh', color='blue', xlabel = 'Order Amount', ylabel = 'Department', title='Deparatment Orders', edgecolor='black', linewidth=1)

##### Observations: 
1. Produce, dairy/eggs, and snacks are the top selling deparatments
2. Bulk, other, and pet items are the lowest selling departments

In [ ]:
# Export bar chart

dep_bar.figure.savefig(os.path.join(path, 'Analysis','Visualizations', 'dep_amounts.png'))

### 8. Customer Profiles

In [ ]:
# Create subset for age, income, department, marriage status, region, and dependants based profiles

df_profiles = df_final_merged[['age', 'income', 'marriage_status', 'region', 'num_dependants', 'department', 'product_name', 'loyalty_flag', 'spending_flag', 'frequency_flag', 'orders_day_of_week', 'order_hour_of_day']]

In [ ]:
df_profiles.head()

In [ ]:
# Find min and max age

df_profiles['age'].agg(['min', 'max'])

In [ ]:
# Create flag for age range: young adult, adult, middle aged, elderly

df_profiles.loc[(df_profiles['age'] >= 18) & (df_profiles['age'] <= 25), 'age_range'] = 'Young adult'
df_profiles.loc[(df_profiles['age'] >= 26) & (df_profiles['age'] <= 39), 'age_range'] = 'Adult'
df_profiles.loc[(df_profiles['age'] >= 40) & (df_profiles['age'] <= 54), 'age_range'] = 'Middle Aged'
df_profiles.loc[df_profiles['age'] >= 55, 'age_range'] = 'Elderly'

In [ ]:
df_profiles.head()

In [ ]:
df_profiles['age_range'].value_counts()

##### Observations:
1. There are more elderly customers (55 and over)
2. Young adults are using IC the least (18 - 25)

In [ ]:
df_profiles.groupby('age_range')['income'].mean()

##### Observations: 
1. Elderly make the most income
2. Adults make the least amount of income

In [ ]:
df_profiles.groupby('age_range')['region'].value_counts()

In [ ]:
df_profiles.groupby('region')['age_range'].value_counts()

In [ ]:
df_profiles.groupby('age_range')['frequency_flag'].value_counts()

In [ ]:
df_profiles.groupby('age_range')['loyalty_flag'].value_counts()

In [ ]:
# Create bar of age to loyalty

age_loyalty = df_profiles.groupby('loyalty_flag')['age_range'].value_counts().unstack()
ax = age_loyalty.plot(kind='bar', stacked=True)

plt.title('Loyalty Distribution by Age Range')
plt.xticks(rotation=0)
plt.ylabel('Customer Count')
plt.legend(title='Age Range', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

In [ ]:
fig = ax.get_figure()

In [ ]:
# Export age_loyalty chart

fig.savefig(os.path.join(path, 'Analysis', 'Visualizations','age_loyalty.png'), dpi=300, bbox_inches='tight')

In [ ]:
df_profiles.groupby('age_range')['spending_flag'].value_counts()

In [ ]:
popular_prod = df_profiles.groupby('age_range')['product_name'].value_counts().groupby(level=0).nlargest(5).reset_index(level=1, drop=True)

In [ ]:
popular_prod

In [ ]:
# Create crosstab of popular_prod
crosstab_top_prod = popular_prod.reset_index().pivot(index='age_range', columns='product_name')

In [ ]:
crosstab_top_prod.to_clipboard()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
top_prods_bar = crosstab_top_prod.plot(kind='bar', color=['red', 'orange', 'yellow', 'green', 'blue'], ax=ax)
plt.title('Top Products by Age')
plt.xlabel(' ')
plt.ylabel('Product Count')
plt.xticks(rotation=0)
plt.legend(['Organic Bananas', 'Banana', 'Organic Baby Spinach', 'Organic Hass Avocado', 'Organic Strawberries'], title='Products', bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# Export top prod chart 

fig.savefig(os.path.join(path, 'Analysis', 'Visualizations', 'top_prods_bar.png'), dpi=300, bbox_inches='tight')

In [ ]:
# Create scatterplot for age and income 

age_income_scatterplot = sns.scatterplot(x='age', y='income', hue='age_range', data=df_profiles)
plt.title('Age vs Income')
plt.xlabel('Age')
plt.ylabel('Income')
plt.legend(title='Age', loc='upper left')

In [ ]:
# Exporting scatterplot for age and income

age_income_scatterplot.figure.savefig(os.path.join(path, 'Analysis', 'Visualizations', 'age_income2.png'))

In [ ]:
df_profiles.groupby('age_range')['num_dependants'].value_counts()

In [ ]:
# Create bar chart of age to num_dependants

fig, ax = plt.subplots(figsize=(12, 6))
top_prods_bar = df_profiles.groupby('age_range')['num_dependants'].value_counts().unstack().plot(kind='bar', color=['red', 'orange', 'yellow', 'green'], ax=ax)

plt.title('Dependants by Age')
plt.xlabel(' ')
plt.ylabel('Number of Dependants')
plt.xticks(rotation=0)
plt.legend(title='Number of Dependants')

In [ ]:
# Export age_dependants chart 

fig.savefig(os.path.join(path, 'Analysis', 'Visualizations', 'age_dependants.png'), dpi=300, bbox_inches='tight')

##### Summary
1. Removed first_name & surname -- privacy reasons
2. New column created -- region from 'states'
3. New column created -- exclusion_flag from 'max_order'
4. New df created -- df_final_exclusion
5. Merge -- df_final_exclusion with df_dep as df_final_merged
6. Dtype change -- df_dep['department_id'] dtype changed to int16 to match df_final_exclusion's
7. df_final_merged shape (30171424, 35)
8. df_final_exclusion shape (30171424, 33)
9. df_final shape (32404859, 33)

### Exports

In [ ]:
# Export df_final_merged 


df_final_merged.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'final_merged_exclusion.pkl'))